# This script is used to remove any node that has no connection

In [11]:
import os
from scipy import sparse
import numpy as np
import pandas as pd
from datasets import *
from utils import *

dataset_name = 'citeseer'
data_dir = os.path.join('dataset/clean', dataset_name)

In [2]:
df_file = os.path.join(data_dir, '{}.train.pkl'.format(dataset_name))
train_df = pd.read_pickle(df_file)

df_file = os.path.join(data_dir, '{}.test.pkl'.format(dataset_name))
test_df = pd.read_pickle(df_file)

In [3]:
def get_num_elements(mat):
    return len(mat.nonzero()[1])

print('num train: {}'.format(len(train_df)))
train_df = train_df[train_df.neighbors.apply(get_num_elements) > 0]
print('(after filtering) num train: {}'.format(len(train_df)))

num train: 1983
(after filtering) num train: 1983


In [40]:
def is_not_self_references(mat):
    if get_num_elements(mat.neighbors) == 1:
        if mat.neighbors.nonzero()[1][0] == mat.name:
            return False
    return True

In [42]:
len(train_df[train_df.apply(axis=1, func=is_not_self_references)])

1945

In [5]:

train_set = TextDataset(dataset_name, data_dir, subset='train')
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=1, shuffle=True)

In [7]:
def get_neighbors(ids, df, max_nodes, batch_size, traversal_func):
    cols = []
    rows = []
    for idx, node_id in enumerate(ids):
        col = traversal_func(df, node_id.item(), max_nodes)
        rows += [idx] * len(col)
        cols += col
    data = [1] * len(cols)
    connections = sparse.csr_matrix((data, (rows, cols)), shape=(batch_size, len(df)))
    return torch.from_numpy(connections.toarray()).type(torch.FloatTensor)

In [43]:
count = 0
for (ids, xb, yb, nb) in train_loader:
    num_nonzero = get_neighbors(ids, train_df, 20, xb.size(0), Random_walk).nonzero().shape[0]
    if num_nonzero <= 0:
        #print(ids)
        count += 1
print(count)

38


In [20]:
train_df.loc[1090].neighbors

<1x1983 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>